In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# The Libraries

In [2]:
!pip install -u -q langchain
!pip install -u -q torch
!pip install -u -q transformers
!pip install -u -q sentence-transformers
!pip install -u -q datasets
!pip install -u -q faiss-cpu
!pip install -u -q  langchain sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━

In [3]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

# Document Loading

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path="/content/drive/MyDrive/combined_data.csv")

data = loader.load()

In [5]:
print(data[18])

page_content='Description: band 1 program bachelor of science degree in software engineering compiled by 1 girma neshir phd assistant prof addis ababa science and technology universityaastu 2 birhanu sisay msc lecturer arba minch universityamu december 2023 ministry of education bonga ethiopia 0 table of contents 1 introduction 2 2 objective of test blueprint 3 3 steps to develop test blueprint 3 4 issues to consider in test blueprint preparation 4 5 required documents to prepare the test blueprint 4 6 i knowledge 4 ii skill 5 iii attitude 6 conclusion 20 1 1 introduction the ministry of education of ethiopia has announced the implementation of exit exam for all undergraduate program students public and private beginning with the 20232024 academic year in order to improve the quality of graduates produced by higher learning institutions the exit exam aimed at checking whether students have acquired the required knowledge skills and attitudes or not to implement this it requires determi

# Document Transformers

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3000,
    chunk_overlap=1000,
    length_function=len,
    is_separator_regex=False,
)

text = text_splitter.split_documents(data)


In [7]:
number_of_chunks = len(text)
print(f"Number of chunks: {number_of_chunks}")

Number of chunks: 158


# Text Embedding


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="distilbert-base-uncased")

In [21]:
t = "This is for test!"

query_result = embeddings.embed_query(t)
query_result[:3]

[0.2629988491535187, -0.29235485196113586, 0.04708665981888771]

# Vector Store

In [22]:
db = FAISS.from_documents(text, embeddings)

In [29]:
question = "What is aastu?"
searchDocs = db.similarity_search(question)
print(searchDocs[1].page_content)

Description: rstanding of the design criteria and determine the optimal location for a chemical pla"


# Preparing the LLM Model

In [30]:
# Import libraries (assuming you already have transformers installed)
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline

# Specify the DistilBERT model name
model_name = "distilbert-base-uncased"  # You can choose a different DistilBERT model from HuggingFace

# Load the tokenizer associated with the DistilBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a question-answering pipeline using the DistilBERT model and tokenizer
question_answerer = pipeline(
    "question-answering",  # Keep the pipeline task the same
    model=model_name,
    tokenizer=tokenizer,
    return_tensors="pt"  # Specify tensor format for processing
)

# Create an instance of the HuggingFacePipeline (optional)
# You can remove this part if you don't need additional model-specific arguments
llm = HuggingFacePipeline(
    pipeline=question_answerer,
    model_kwargs={"temperature": 0.7, "max_length": 512},  # Optional arguments
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Retriever

In [31]:
retriever = db.as_retriever()


In [36]:
docs = retriever.get_relevant_documents("What is AASTU?")
print(docs[2].page_content)

cm1 alkane 2920 cm1 nitrile 2114 cm1 and aromatics 1613 cm1 the minimum cr adsorption performance of 156 was obtained whereas maximum removal of 904 was recorded at the experimental condition of ph 2 adsorbent dose of 15 g100 ml and contact time of 120 min at a fixed value of initial cr concentration of 100 mgl similarly the maximum cr removal from real electroplating wastewater was 812 at this optimum point langmuirs model best described the experimental value at r2 096 which implies the adsorption is chemically bonded homogeneous and monolayer pseudosecondorder model best fits with the experimental data with r2 value of 099 the adsorbent was regenerated for seven cycles and the removal efficiency decreased from 9325 to 2135 finally this technology is promising to be scaled up to an industrial level 10 issue 1 september 2023 in honor of tarekegn berhanu esho phd associate professor of analytical chemistry june 1963october 2023 it is with heavy hearts and a deep sense of sor row that w

# RetrievalQA Chain

In [37]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 4})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)